In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import brier_score_loss
from numpy import std, mean, sqrt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# read csv files into dataframes
workers = pd.read_csv('datasets/participants.csv')
results = pd.read_csv('datasets/results.csv')
sample = pd.read_csv('datasets/defendants.csv')

### Performance in the Presence of Risk Assessment Model False Predictions
- Does the introduction of explanations improve participants' performance when the risk assessment model predicts a false positive or false negative?
- Across predictions with explanations & false positive/negative risk assesssment scores, regress participant prediction scores against participant treatment controlling for risk asssessment performance, participant demographic information, and defendant features -> **positive and statistically significant association only between participant prediction score and the diverse counterfactual treatment -> improved performance in only one treatment**

In [3]:
# get defendants with false positive and false negative risk assessment model predictions
sample_err = list(sample.loc[((sample['real_outcome']==1) & (sample['alg_outcome']==0)) | ((sample['real_outcome']==0) & (sample['alg_outcome']==1))]['id'])

In [4]:
# get predictions about those defendants
results_err = results.loc[(results['defendant_id'].isin(sample_err)) & (results['treatment']!=0)]

In [5]:
# get risk assessment model and participant brier scores for each of the predictions
results_err['ra_brier_score'] = np.nan
results_err['participant_brier_score'] = np.nan
for index, row in results_err.iterrows():
    results_err.at[index,'ra_brier_score'] = 1 -   brier_score_loss([row['actual_outcome']], [row['ra_score'] / 10])
    results_err.at[index,'participant_brier_score'] = 1 -   brier_score_loss([row['actual_outcome']], [row['participant_score'] / 10])

In [6]:
cols = ['participant_degree', 'participant_age']
for col in cols:
    col_zscore = col + '_zscore'
    results_err[col_zscore] = (results_err[col] - results_err[col].mean())/results_err[col].std(ddof=0)

In [7]:
# run linear mixed-effects model with random effects for participant and defendant identities
results_err["group"] = 1
vcf = {"session_id": "0 + C(session_id)", "defendant_id": "0 + C(defendant_id)"}
md = smf.mixedlm('participant_brier_score~C(treatment) + ra_brier_score + defendant_age + C(defendant_sex) + C(defendant_race) + defendant_priors + C(defendant_charge_degree) + C(defendant_offense_type) + participant_age_zscore + participant_degree_zscore + C(participant_gender) + C(participant_politics) + C(participant_ethnicity)',data=results_err, vc_formula=vcf, groups=results_err['group'])

In [8]:
mdf = md.fit()
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Mixed Linear Model Regression Results
=================================================================================================
Model:                      MixedLM          Dependent Variable:          participant_brier_score
No. Observations:           4512             Method:                      REML                   
No. Groups:                 1                Scale:                       0.0447                 
Min. group size:            4512             Log-Likelihood:              155.3440               
Max. group size:            4512             Converged:                   No                     
Mean group size:            4512.0                                                               
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                               0.200    0.116  1.726 0.084 -0.027  0.427
C(treatment)[T.2]                                       0.046    0.017  2.664 0.008  0.012  0.080
C(treatment)[T.3]                                       0.005    0.017  0.318 0.750 -0.028  0.039
C(treatment)[T.4]                                       0.004    0.017  0.236 0.813 -0.029  0.037
C(treatment)[T.5]                                      -0.002    0.017 -0.138 0.890 -0.036  0.031
C(defendant_sex)[T.male]                               -0.028    0.038 -0.732 0.464 -0.102  0.046
C(defendant_race)[T.Caucasian]                          0.006    0.034  0.165 0.869 -0.061  0.072
C(defendant_charge_degree)[T.M]                         0.015    0.043  0.359 0.720 -0.068  0.099
C(defendant_offense_type)[T.drug crime]                 0.039    0.061  0.640 0.522 -0.081  0.159
C(defendant_offense_type)[T.fraud and financial crime] -0.067    0.160 -0.421 0.674 -0.380  0.246
C(defendant_offense_type)[T.obstruction of justice]     0.018    0.080  0.223 0.823 -0.139  0.175
C(defendant_offense_type)[T.property crime]             0.046    0.066  0.692 0.489 -0.084  0.175
C(defendant_offense_type)[T.public order crime]         0.052    0.065  0.807 0.420 -0.075  0.180
C(defendant_offense_type)[T.violent crime]              0.021    0.060  0.346 0.729 -0.096  0.137
C(defendant_offense_type)[T.weapons-related crime]      0.043    0.103  0.422 0.673 -0.158  0.245
C(participant_gender)[T.male]                           0.015    0.011  1.390 0.165 -0.006  0.037
C(participant_politics)[T.Don't know]                   0.302    0.124  2.431 0.015  0.058  0.545
C(participant_politics)[T.Independent]                 -0.026    0.015 -1.770 0.077 -0.054  0.003
C(participant_politics)[T.Republican]                  -0.023    0.014 -1.705 0.088 -0.050  0.003
C(participant_politics)[T.Something else]              -0.079    0.040 -1.971 0.049 -0.158 -0.000
C(participant_ethnicity)[T.Asian]                      -0.053    0.027 -1.988 0.047 -0.106 -0.001
C(participant_ethnicity)[T.Caucasian]                  -0.020    0.016 -1.228 0.219 -0.051  0.012
C(participant_ethnicity)[T.Latino/hispanic]            -0.008    0.031 -0.275 0.783 -0.069  0.052
C(participant_ethnicity)[T.Native American]             0.073    0.033  2.223 0.026  0.009  0.138
C(participant_ethnicity)[T.Other/unknown]              -0.028    0.059 -0.473 0.636 -0.142  0.087
C(participant_ethnicity)[T.Two or more]                -0.041    0.043 -0.950 0.342 -0.126  0.044
ra_brier_score                                          0.668    0.125  5.334 0.000  0.422  0.913
defendant_age                                           0.000    0.001  0.043 0.966 -0.003  0.003
defendant_priors                                       -0.005    0.005 -1.042 0.297 -0.014  0.004
participant_age_zscore                                 -0.010

### Explanation Informativeness' Correlation with Performance
- Is participant self-reported explanation informativeness positively correlated with (a) participant performance, and (b) participant self-reported confidence in explaining how they arrived at their predictions? 
- Within each treatment, regress explanation informativeness against survey answers for (a) and (b) controlling for risk assessment model performance, participant performance, demographic information, and exit survey responses -> **postive and statistically significant between participants' self-reported explanation usefulness and ability to explain their decision-making process only in the selective feature attribution treatment**

In [9]:
workers_exp = workers.loc[workers['treatment']==5]

In [10]:
cols = ['participant_degree', 'participant_age']
for col in cols:
    col_zscore = col + '_zscore'
    workers_exp[col_zscore] = (workers_exp[col] - workers_exp[col].mean())/workers_exp[col].std(ddof=0)

In [11]:
y = workers_exp['self_reported_exp_usefulness'] 
X = workers_exp[['self_reported_exp_ability','participant_brier_score', 'self_reported_influence', 'self_reported_ra_accuracy','accountability', 'relative_confidence', 'self_reported_ra_fairness', 'confidence', 'ra_brier_score', 'participant_gender', 'participant_age_zscore', 'participant_degree_zscore', 'participant_ethnicity', 'participant_politics', 'ml_fam', 'cj_fam']] 

In [12]:
xdum = pd.get_dummies(X, columns=['participant_gender', 'participant_ethnicity', 'participant_politics'], drop_first=True)

In [13]:
# fit x and y with an OLS regression
res = sm.OLS(y, xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      OLS Regression Results                                     
=================================================================================================
Dep. Variable:     self_reported_exp_usefulness   R-squared (uncentered):                   0.955
Model:                                      OLS   Adj. R-squared (uncentered):              0.941
Method:                           Least Squares   F-statistic:                              65.09
Date:                          Tue, 03 Aug 2021   Prob (F-statistic):                    2.33e-38
Time:                                  19:55:17   Log-Likelihood:                         -103.42
No. Observations:                            93   AIC:                                      252.8
Df Residuals:                                70   BIC:                                      311.1
Df Model:                                    23                                                  
Covariance Type:                      nonrobust                                                  
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
self_reported_exp_ability                 0.2605      0.126      2.061      0.043       0.008       0.513
participant_brier_score                   2.8033      1.836      1.527      0.131      -0.859       6.465
self_reported_influence                   0.1097      0.118      0.934      0.354      -0.125       0.344
self_reported_ra_accuracy                 0.0687      0.147      0.469      0.641      -0.224       0.361
accountability                            0.0081      0.108      0.075      0.940      -0.207       0.223
relative_confidence                       0.0428      0.197      0.217      0.829      -0.350       0.436
self_reported_ra_fairness                 0.1524      0.135      1.132      0.261      -0.116       0.421
confidence                                0.2633      0.150      1.758      0.083      -0.035       0.562
ra_brier_score                           -2.1210      1.935     -1.096      0.277      -5.979       1.737
participant_age_zscore                   -0.3368      0.111     -3.033      0.003      -0.558      -0.115
participant_degree_zscore                -0.1550      0.103     -1.501      0.138      -0.361       0.051
ml_fam                                    0.0107      0.124      0.087      0.931      -0.236       0.258
cj_fam                                    0.1111      0.146      0.758      0.451      -0.181       0.403
participant_gender_Male                  -0.6098      0.203     -2.998      0.004      -1.015      -0.204
participant_ethnicity_Asian              -0.0247      0.401     -0.062      0.951      -0.825       0.776
participant_ethnicity_Caucasian          -0.3727      0.283     -1.318      0.192      -0.937       0.191
participant_ethnicity_Native American    -2.1714      0.967     -2.245      0.028      -4.100      -0.243
participant_ethnicity_Other/unknown       0.0694      0.871      0.080      0.937      -1.668       1.806
participant_ethnicity_Two or more        -0.0552      1.017     -0.054      0.957      -2.084       1.974
participant_politics_Don't know           0.2278      0.984      0.232      0.818      -1.734       2.189
participant_politics_Independent          0.1779      0.243      0.732      0.467      -0.307       0.663
participant_politics_Republican           0.5356      0.302      1.772      0.081      -0.067       1.139
participant_politics_Something else      -1.1262      0.581     -1.940      0.056      -2.284       0.032
==============================================================================
Omnibus:                        1.430  